<a href="https://colab.research.google.com/github/PyBlin/Study/blob/main/PyML/Chapter3_Evaluation/Chap3_1_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 분류의 성능 평가 지표( Evaluation Metric )
>* 정확도( Accuracy )
>* 오차행렬( Confusion Matrix )
>* 정밀도( Precision )
>* 재현율( Recall )
>* F1 Score
>* ROC-AUC( Receiver Operating Characteristic - Area Under the Curve )

* 타이타닉 ML 알고리즘의 예측 정확도가 보통 80%대였습니다.
* 하지만, 여자인 경우에 생존률이 높았기 때문에 무조건 여자만 생존으로 예측해도 결과는 비슷하게 나올 수 있습니다.
* 그저 성별 조건 하나만으로 결정하는 별거 아닌 알고리즘도 높은 정확도를 나타내는 상황이 발생하게 됩니다.

# 1.Accuracy

* acc = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수

In [1]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass
    
    # predict() : Sex==1 --> 0, Sex!=1 --> 0
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))

        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred

* BaseEstimator 클래스를 상속받아 아무런 학습을 하지 않고, 성별만으로 생존자를 예측하는 단순 Classifier를 생성합니다.
* Chapter2에서 설정했던 transform_features()을 해줍니다.

In [2]:
# transform_features() 생성

from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def titanic_fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# ML 알고리즘에 불필요한 속성 제거
def titanic_drop(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# label encoding
def titanic_le(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 위에서 설정한 데이터 전처리 함수 종합
def transform_features(df):
    df = titanic_fillna(df)
    df = titanic_drop(df)
    df = titanic_le(df)
    return df

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터 로딩, 데이터 가공, 학습/테스트 데이터 분할
titanic_df = pd.read_csv('./train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

# transform_features() 설정 필요
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, 
                                                    test_size=0.2, 
                                                    random_state=0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypred = myclf.predict(X_test)
print(f"Dummy Classifier's accuracy : {accuracy_score(y_test, mypred):.4f}")

Dummy Classifier's accuracy : 0.7877


* 위와 같은 단순한 알고리즘으로 예측을 해도 정확도가 78.77%로 꽤 높게 나옵니다.
* 그래서 정확도를 평가 지표로 사용할 때는 매우 신중해야 합니다.
* 이제 MNIST 데이터셋의 '7'만 True로 하여 나머지를 0으로 예측해보겠습니다.

In [4]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass

    # 입력으로 들어오는 X 데이터셋의 크기만큼 모두 0값으로 변환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

# 사이킷런 내장 데이터셋을 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits '7'이면 True, astype(int) --> 1, 나머지 0
y = (digits.target==7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, 
                                                    random_state=11)

* 즉, False(0)가 90% 이므로 단순하게만 예측해도 결과는 90%입니다.

In [5]:
# 불균형한 레이블 데이터 분포도 확인
print(f"레이블 테스트셋 크기 : {y_test.shape}")
print(f"\n테스트셋 레이블 0과 1의 분포도 : \n{pd.Series(y_test).value_counts()}")

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print(f"\n모두 0으로 예측한 정확도 : {accuracy_score(y_test, fakepred):.4f}")

레이블 테스트셋 크기 : (450,)

테스트셋 레이블 0과 1의 분포도 : 
0    405
1     45
dtype: int64

모두 0으로 예측한 정확도 : 0.9000


* 이런 단순 예측이 ML 알고리즘과 맞설 수 있다는 것은 말도 안됩니다.
* 이처럼 정확도 평가 지표는 불균형한 레이블 데이터셋에서는 성능 수치로 사용돼서는 안됩니다.
* 여러 가지 분류 지표와 함께 적용해야 합니다.